In [ ]:
# Add project src to path.
import set_path

import time
import numpy as np
import glob

from src.region_growing.label_connected_comp import LabelConnectedComp
from src.utils.las_utils import read_las
from src.utils.labels import Labels
from src.utils.csv_utils import write_csv

In [ ]:
# Original pointcloud data folder.
orig_pc_folder = '../datasets/pointcloud/'
# Predicted pointcloud labels data folder.
pred_pc_folder = '../datasets/pointcloud/pred/'
# Predicted pointcloud labels data folder.
pole_locations_path = '../datasets/lamp_post_locations.csv'

In [ ]:
# We try to ignore the false positive lamp posts using these settings
min_component_size = 500
max_component_size = 10000
min_component_height = 4

def get_cluster_dim(point_components, mask, points):
    """
    Get the dimensions of a cluster and return the coordinates
    of the middlepoint.
    """
    mask_indices = np.where(mask)[0]

    cc_labels, counts = np.unique(point_components,
                                  return_counts=True)

    cc_labels_filtered = cc_labels[(counts >= min_component_size) 
                                   & (counts <= max_component_size)]

    xy_locations_per_tile = []
    for cc in cc_labels_filtered:
        # select points that belong to the cluster
        cc_mask = (point_components == cc)

        xyz = points[mask_indices[cc_mask]]
        x_points = xyz[:,0]
        y_points = xyz[:,1]
        z_points = xyz[:,2]
        z_min, z_max = np.min(z_points), np.max(z_points)

        if (z_max - z_min) > min_component_height:
            # Calculate the centroid of the cluster
            centroid = (round(sum(x_points) / len(xyz),2), 
                        round(sum(y_points) / len(xyz),2))
            xy_locations_per_tile.append(centroid)
    
    return xy_locations_per_tile

In [ ]:
xy_locations = []

for tile in glob.glob(orig_pc_folder + "*.laz"):
    tilename = tile.split("/")[-1]
    pointcloud = read_las(tile)
    pointcloud_pred = read_las(pred_pc_folder + tilename)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = pointcloud_pred.label   

    mask = labels == Labels.STREET_LIGHT
    
    lcc = LabelConnectedComp(Labels.STREET_LIGHT)
    point_components = lcc.get_components(points[mask])
    
    xy_locations_per_tile = get_cluster_dim(point_components, mask, points)
    
    xy_locations.extend(xy_locations_per_tile)
    
write_csv(pole_locations_path, xy_locations, csv_headers=['rd_x', 'rd_y'])